In [4]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
# from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

ModuleNotFoundError: No module named 'pandas'

In [4]:
utah = pd.read_csv("UTAH_25Gbps_ssd_merged.csv")
wiscon_hdd = pd.read_csv("WISCONSIN_10Gbps_hdd_merged.csv")
wiscon_sdd = pd.read_csv("WISCONSIN_10Gbps_ssd_merged.csv")
hpcn_ssd = pd.read_csv("hpcn_1Gbps_ssd_merged.csv")

In [5]:
df = wiscon_sdd
np.round((sum(sum((df == 0).values))) / (len(df) * len(df.columns)),3)

0.331

In [6]:
y = df.label_value
X = df.drop(columns="label_value") # df[features]
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(X_train.shape)
X_train, y_train = RandomUnderSampler(sampling_strategy="all").fit_resample(X_train, y_train)
print(X_train.shape)

(6408, 116)
(1422, 116)


In [7]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
print(metrics.confusion_matrix(y_test, y_pred))

97.38
[[ 23   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 415   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  97   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  49   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  85   0   0  23   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1  99   0   4   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  14  12   2 187   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  77   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 102   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 188   0   0   0   0   0]
 [  0   0   0   0  

In [8]:
score = np.round(clf.feature_importances_, 2)
print("Cummulative Importance Score: \n", np.cumsum(sorted(list(score), reverse=True)))

from collections import Counter
print(Counter(score))

Cummulative Importance Score: 
 [0.06 0.12 0.16 0.2  0.24 0.27 0.3  0.33 0.36 0.39 0.41 0.43 0.45 0.47
 0.49 0.51 0.53 0.55 0.57 0.59 0.61 0.63 0.64 0.65 0.66 0.67 0.68 0.69
 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83
 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97
 0.98 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99
 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99
 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99
 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99
 0.99 0.99 0.99 0.99]
Counter({0.0: 58, 0.01: 36, 0.02: 12, 0.03: 5, 0.04: 3, 0.06: 2})


In [9]:
features = []
for i in range(len(score)):
    if score[i] >= 0.02:
        features.append(X.columns[i])
        print(df.columns[i])

sender_cwnd_rate
sender_retrans
sender_segs_in
sender_avg_send_value
sender_send_buffer_value
sender_write_bytes_io
sender_tcp_snd_buffer_max
sender_req_waittime
sender_req_active
sender_ost_read
sender_avg_waittime_md
sender_system_cpu_percent
sender_system_memory_percent
sender_remote_ost_read_bytes
receiver_seg_out
receiver_segs_in
receiver_req_active
receiver_ost_read
receiver_ost_write
receiver_avg_waittime_md
receiver_system_cpu_percent
receiver_system_memory_percent


In [20]:
np.round(df.groupby("label_value").receiver_avg_waittime_md.agg(["mean", "std"]),2)

,mean,std
label_value,,
0,414.35,0.48
1,414.98,0.15
17,414.06,0.46
33,414.69,0.46
34,415.00,0.00
35,415.91,0.28
36,417.84,1.06
40,417.92,1.12
44,417.94,1.13


In [291]:
y_new = df.label_value
X_new = df[features] #.drop(columns="label_value")
X_train, X_test, y_train, y_test = train_test_split(X_new,y_new)
print(X_train.shape)
X_train, y_train = RandomUnderSampler(sampling_strategy="all").fit_resample(X_train, y_train)
print(X_train.shape)

(6408, 22)
(1260, 22)


In [292]:
clf = RandomForestClassifier(n_estimators=100)
# clf = make_pipeline(StandardScaler(), SVC())
# clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
# print(metrics.confusion_matrix(y_test, y_pred))

97.71


In [293]:
y_utah = utah.label_value
X_utah = utah[features]
y_pred = clf.predict(X_utah) #.drop(columns="label_value")
print(np.round(metrics.accuracy_score(y_utah, y_pred) * 100, 2))
print(metrics.confusion_matrix(y_utah, y_pred))

36.59
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  109    0    0]
 [   0  661    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0 1394    0    0    0    0    0    0    0    0    0    0    0
     0  374    0    0]
 [   0    0    0    1    0    0    0    0    0    0    0    0    0    0
     0  409    0    0]
 [   0    0    0    0  109    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0   51    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0   10    0    0    1    0    0    0   11
     0  416    0    0]
 [   0    0    0    0    0    0    0    6    0    3    0    0    0    2
     0  428    0    1]
 [   0    0    0    1    0    0    0    0    0    0    0    0    0    0
     2  435    0    0]
 [   0    0    0    0    0    0    0    1    0   13    0    0    0    2
     6  856    0    1]
 [   0    0    0    0    0    0    0    0   

In [294]:
y_utah = wiscon_hdd.label_value
X_utah = wiscon_hdd[features]
y_pred = clf.predict(X_utah) #.drop(columns="label_value")
print(np.round(metrics.accuracy_score(y_utah, y_pred) * 100, 2))
# print(metrics.confusion_matrix(y_utah, y_pred))

67.12


In [295]:
y_utah = wiscon_sdd.label_value
X_utah = wiscon_sdd[features]
y_pred = clf.predict(X_utah) #.drop(columns="label_value")
print(np.round(metrics.accuracy_score(y_utah, y_pred) * 100, 2))

97.76


In [226]:
df.groupby("label_value").sender_avg_reord_seen.mean()

label_value
0     0.0
1     0.0
17    0.0
33    0.0
34    0.0
35    0.0
36    0.0
40    0.0
44    0.0
48    0.0
59    0.0
62    0.0
66    0.0
74    0.0
82    0.0
86    0.0
87    0.0
88    0.0
Name: sender_avg_reord_seen, dtype: float64